In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.getcwd()
os.chdir('drive')
os.chdir('MyDrive')
os.chdir('Coursera Project')

Mounted at /content/drive


In [ ]:
# Importing Necessary Libraries
import pandas as pd

In [ ]:
# Loading the transformed data
Company_names = ['Amazon', 'Apple', 'Google', 'Meta', 'Netflix']
company_dict = {}
for i in Company_names:
  company_dict[i]=pd.read_csv(i+'_df.csv').set_index('Date')

In [ ]:
# Importing Necessary Libraries
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Function to get the Train, and Test data
def construct_feature_target(df_in):
  df = df_in
  sc = MinMaxScaler(feature_range=(0,1))
  df_in = sc.fit_transform(df_in)
  X = df_in[:,:-1]
  y = df_in[:,-1:]
  X = np.reshape(X,(X.shape[0],X.shape[1]))
  y = np.reshape(y,(y.shape[0],y.shape[1]))
  print(X.shape,y.shape)
  return df.index[0:int(0.8*len(df_in))], X[0:int(0.8*len(df_in))],y[0:int(0.8*len(df_in))], df.index[int(0.8*len(df_in)):], X[int(0.8*len(df_in)):], y[int(0.8*len(df_in)):], sc

In [ ]:
# Getting the Train and Test data for each company and saving it to a dictionary
df_train = {}
X_train = {}
y_train = {}
df_val = {}
X_val = {}
y_val = {}
df_test = {}
X_test = {}
y_test = {}
sc = {}
for i in Company_names:
  df_train[i], X_train[i], y_train[i], df_test[i],  X_test[i], y_test[i], sc[i]= construct_feature_target(company_dict[i])

(6464, 10) (6464, 1)
(10624, 10) (10624, 1)
(4608, 10) (4608, 1)
(2688, 10) (2688, 1)
(5184, 10) (5184, 1)


In [ ]:
# Printing the shape Train and Test data for each company
for i in Company_names:
  print(X_train[i].shape, y_train[i].shape, X_test[i].shape, y_test[i].shape)

(5171, 10) (5171, 1) (1293, 10) (1293, 1)
(8499, 10) (8499, 1) (2125, 10) (2125, 1)
(3686, 10) (3686, 1) (922, 10) (922, 1)
(2150, 10) (2150, 1) (538, 10) (538, 1)
(4147, 10) (4147, 1) (1037, 10) (1037, 1)


In [ ]:
# Importing XGBoost Library
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
# Setting up a grid search for the XGBoost model and fiting the model on the training data 
xgb_reg = XGBRegressor()
hyper_parameters = {'objective':['reg:squarederror'],
              'learning_rate': [0.01, 0.03, 0.05, 0.07, 0.09], 
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}

xgb_grid = {}
for i in Company_names:
  xgb_grid[i] = GridSearchCV(xgb_reg,
                          hyper_parameters,
                          cv = 2,
                          n_jobs = 5,
                          verbose=True)

  xgb_grid[i].fit(X_train[i], y_train[i])

Fitting 2 folds for each of 15 candidates, totalling 30 fits
[03:10:12] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

Fitting 2 folds for each of 15 candidates, totalling 30 fits
[03:10:44] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

Fitting 2 folds for each of 15 candidates, totalling 30 fits
[03:11:14] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

Fitting 2 folds for each of 15 candidates, totalling 30 fits
[03:11:38] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

Fitting 2 folds for each of 15 candidates, totalling 30 fits
[03:12:03] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.



In [ ]:
# Getting the prdictions of the XGBoost model on Test data
for i in Company_names:
  xgb_grid[i].predict(X_test[i])

In [ ]:
# Importing Necessary Libraries
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# Getting a dataframe for important performance indicators for the XGBoost Regression model
metrics_list = ["Mean_Absolute_Error_Train", "Mean_Absolute_Error_Test",
                "Mean_Squared_Error_Train", "Mean_Squared_Error_Test",
                "R2_Score_Train", "R2_Score_Test",]
metrics = np.ones((5,6)).astype(np.float32)
for i in range(len(Company_names)):
  metrics[i][0] = mean_absolute_error(y_train[Company_names[i]],xgb_grid[Company_names[i]].predict(X_train[Company_names[i]]))
  metrics[i][2] = mean_squared_error(y_train[Company_names[i]],xgb_grid[Company_names[i]].predict(X_train[Company_names[i]]))
  metrics[i][4] = r2_score(y_train[Company_names[i]],xgb_grid[Company_names[i]].predict(X_train[Company_names[i]]))
  metrics[i][1] = mean_absolute_error(y_test[Company_names[i]],xgb_grid[Company_names[i]].predict(X_test[Company_names[i]]))
  metrics[i][3] = mean_squared_error(y_test[Company_names[i]],xgb_grid[Company_names[i]].predict(X_test[Company_names[i]]))
  metrics[i][5] = r2_score(y_test[Company_names[i]],xgb_grid[Company_names[i]].predict(X_test[Company_names[i]]))

In [ ]:
# Printing the dataframe for important performance indicators for the XGBoost Regression model
df = pd.DataFrame(metrics, columns=metrics_list)
df["Company"] = Company_names
df = df.set_index("Company")
df

,Mean_Absolute_Error_Train,Mean_Absolute_Error_Test,Mean_Squared_Error_Train,Mean_Squared_Error_Test,R2_Score_Train,R2_Score_Test
Company,,,,,,
Amazon,0.000435,0.293438,4.322876e-07,0.121234,0.999909,-2.451219
Apple,0.000166,0.264854,1.061230e-07,0.143941,0.999893,-0.945502
Google,0.000614,0.257838,7.387299e-07,0.102264,0.999936,-1.766176
Meta,0.004520,0.066191,4.362557e-05,0.009997,0.998754,0.820247
Netflix,0.000417,0.091933,4.814652e-07,0.020096,0.999968,0.376608


In [ ]:
# Saving the dataframe for important performance indicators for the XGBoost Regression model
df.to_csv("ML_Model_Evaluation.csv")